In [ ]:
import os
import sys
import math
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


sys.path.append('../..')
import model.Baseline as baseline
import model.my_model as mymodel
import model.util_loss as util_ls
import model.util_dataloader as util_dl
import model.util_model as util_md
import data_process.util_data as util_dt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")

# local
file_save_dir = r'C:\Users\29492\Desktop\exp12'
file_load_dir = r'D:\dataset\DataFrames\REDD'
# server
# file_save_dir = r'../../data'
# file_load_dir = r'../../data'
dataset_name = 'REDD'
file_code = '0103'

dm = util_md.DictManager(file_save_dir)
mwm = util_md.ModelWeightManager(file_save_dir)

In [ ]:
dataframe_path = os.path.join(file_load_dir, r'house_1.csv')
df = pd.read_csv(dataframe_path)
df.set_index('time', inplace=True)
# df.describe(percentiles=[0.10,0.25,0.50,0.75,0.8,0.85,0.9,0.95])


house_1_cols = ['oven-3', 'oven-4', 'refrigerator-5', 'dishwaser-6', 'lighting-9',
                'washer_dryer-10', 'microwave-11', 'bathroom_gfi-12', 'electric_heat-13', 'stove-14',
                'lighting-17', 'lighting-18', 'washer_dryer-19', 'washer_dryer-20']
filtered_columns = df[house_1_cols]
# filtered_columns.describe(percentiles=[0.10,0.25,0.50,0.6,0.75,0.8,0.85,0.9,0.95])


apps_name = filtered_columns.columns.to_list()
print(apps_name)


filtered_columns['sum'] = filtered_columns.sum(axis=1)
norm_data, min_val, max_val = util_dt.normalization_interval(filtered_columns.to_numpy(), 0, 1)
main = norm_data[:,-1]
apps = norm_data[:,:-1]
print(f'min: {min_val}; max: {max_val}')
print(np.min(main), np.max(main))
print(np.min(apps), np.max(apps))
util_dt.print_shape(main, apps)


on_threshold = [50, 50, 50, 50, 20, 50, 50, 50, 50, 50, 20, 20, 50, 50]
sliding_window_len = 599

In [ ]:
in_seq_l = sliding_window_len
out_dim = len(apps_name)
print(f'in_seq_l: {in_seq_l}; out_dim: {out_dim}')

loss_factor=max_val-min_val
print(f'loss_factor:{loss_factor}')


metric_dict_one2all = {'MAE(apps)':util_ls.MAE(single_class=False),
                       'MAE(mean)':util_ls.MAE(single_class=True),
                'SAE(apps)':util_ls.SignalAggregateError(single_class=False, period_len=450),
                'SAE(mean)':util_ls.SignalAggregateError(single_class=True, period_len=450),
                'MAE(offon)':util_ls.MAE_off_on(single_class=False)}


w_main, w_apps = util_dt.generate_window_samples(input_1=main, input_2=apps, window_size_1=sliding_window_len,
                                     window_size_2=1, offset=math.floor(sliding_window_len/2))
w_main = np.expand_dims(w_main, axis=1)
w_apps_sum = w_apps.sum(axis=-1)
w_apps_sum = np.expand_dims(w_apps_sum, axis=1)
w_apps_on = (w_apps>(on_threshold-min_val)/(max_val-min_val)).astype('int')
util_dt.print_shape(w_main, w_apps, w_apps_sum, w_apps_on)

In [ ]:
b_s = 450*60
eval_b_s = 450*60

# SS (strong supervision)

## Baseline

In [ ]:
exp_num = 3
epoch_n = 20

dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))

subset_1, subset_2 = util_dl.split_Dataset_orderly(dataset, [0.8,0.2])
trainset, valset = util_dl.split_to_DataLoader_randomly(dataset=subset_1, split_ratio=[0.75,0.25], batch_size=b_s, shuffle_flag=True)
testset = util_dl.just_to_DataLoader(dataset=subset_2, batch_size=b_s, shuffle_flag=False)
del dataset, subset_1, subset_2

for i in range(exp_num):
    model = mymodel.AblationMark_baseline(in_seq_l, out_dim)
    score_train_vali = util_md.train_val(device, model, epoch_n, trainset, valset, loss_factor=loss_factor,
                                         return_metric_dict=metric_dict_one2all, decay_params=[5,0.5])
    score_test = util_md.train_val(device, model, valset=testset, loss_factor=loss_factor,
                                         return_metric_dict=metric_dict_one2all)
    total_record = {'train_vali':score_train_vali, 'test':score_test}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset=dataset_name,model=type(model).__name__
                                                           ,appname='all',date=f'{file_code}-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)

# WS (weak supervision)

In [ ]:
dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                            torch.from_numpy(w_apps_on).to(torch.float32),
                            torch.from_numpy(w_apps_sum).to(torch.float32),
                            torch.from_numpy(w_apps).to(torch.float32))
dataset.describe()
subset_1, subset_2 = util_dl.split_Dataset_orderly(dataset, [0.8,0.2])
trainset, valset = util_dl.split_to_DataLoader_randomly(dataset=subset_1, split_ratio=[0.75,0.25],
                                                        batch_size=b_s, shuffle_flag=True)
testset = util_dl.just_to_DataLoader(dataset=subset_2, batch_size=eval_b_s, shuffle_flag=False)
del dataset, subset_1, subset_2

## WS

In [ ]:
exp_num = 3
epoch_n = 50

for i in range(exp_num):
    model = mymodel.AblationMark_Ws(in_seq_l, out_dim)
    score_train_vali = util_md.train_val_Mark(device, model, epoch_n, trainset, valset, loss_factor=loss_factor,
                                         return_metric_dict=metric_dict_one2all, decay_params=[10,0.5])
    score_test = util_md.train_val_Mark(device, model, valset=testset, loss_factor=loss_factor,
                                         return_metric_dict=metric_dict_one2all)
    total_record = {'train_vali':score_train_vali, 'test':score_test}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset=dataset_name,model=type(model).__name__
                                                           ,appname='all',date=f'{file_code}-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)

## WS+MTL

In [ ]:
exp_num = 10
epoch_n = 50·

for i in range(exp_num):
    model = mymodel.AblationMark_Ws_Mtl(in_seq_l, out_dim)
    score_train_vali, score_train_vali_clas = util_md.train_val_Mark_DoubleTask(device, model, epoch_n, trainset, valset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all, decay_params=[10,0.5])
    score_test, score_test_clas = util_md.train_val_Mark_DoubleTask(device, model, valset=testset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all)
    total_record = {'train_vali':score_train_vali, 'test':score_test,
                    'train_vali_clas':score_train_vali_clas, 'test_clas':score_test_clas}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset=dataset_name,model=type(model).__name__
                                                           ,appname='all',date=f'{file_code}-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)

# Single-task learning    state classification

In [ ]:
exp_num = 3
epoch_n = 20


dataset = util_dl.VariableDataset(torch.from_numpy(w_main).to(torch.float32),
                                      torch.from_numpy(w_apps_on).to(torch.float32))

subset_1, subset_2 = util_dl.split_Dataset_orderly(dataset, [0.8,0.2])
trainset, valset = util_dl.split_to_DataLoader_randomly(dataset=subset_1, split_ratio=[0.75,0.25], batch_size=b_s, shuffle_flag=True)
testset = util_dl.just_to_DataLoader(dataset=subset_2, batch_size=b_s, shuffle_flag=False)
del dataset, subset_1, subset_2


return_metric_clas_dict={'Acc(apps)':util_ls.LogitAccuracy(reduction='none'), 'Acc(mean)':util_ls.LogitAccuracy()}

for i in range(exp_num):
    model = mymodel.AblationMark_classification(in_seq_l, out_dim)
    score_train_vali = util_md.train_val(device, model, epoch_n, trainset, valset, loss_fn=nn.BCEWithLogitsLoss(),
                                         return_metric_dict=return_metric_clas_dict, decay_params=[5,0.5])
    score_test = util_md.train_val(device, model, valset=testset, loss_fn=nn.BCEWithLogitsLoss(),
                                         return_metric_dict=return_metric_clas_dict)
    total_record = {'train_vali':score_train_vali, 'test':score_test}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset=dataset_name,model=type(model).__name__
                                                           ,appname='all',date=f'{file_code}-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)

# WS+MTL+ALW

In [ ]:
exp_num = 10
epoch_n = 50

for i in range(exp_num):
    model = mymodel.AblationMark_Ws_Mtl_Alw(in_seq_l, out_dim)
    score_train_vali, score_train_vali_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, epoch_n, trainset, valset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all, decay_params=[10,0.5])
    score_test, score_test_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, valset=testset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all)
    total_record = {'train_vali':score_train_vali, 'test':score_test,
                    'train_vali_clas':score_train_vali_clas, 'test_clas':score_test_clas}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset=dataset_name,model=type(model).__name__
                                                           ,appname='all',date=f'{file_code}-AWL2L-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)

In [ ]:
exp_num = 10
epoch_n = 50

for i in range(exp_num):
    model = mymodel.AblationMark_Ws_Mtl_Alw(in_seq_l, out_dim)
    score_train_vali, score_train_vali_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss_1L(device, model, epoch_n, trainset, valset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all, decay_params=[10,0.5])
    score_test, score_test_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss_1L(device, model, valset=testset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all)
    total_record = {'train_vali':score_train_vali, 'test':score_test,
                    'train_vali_clas':score_train_vali_clas, 'test_clas':score_test_clas}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset=dataset_name,model=type(model).__name__
                                                           ,appname='all',date=f'{file_code}-AWL1L-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)

## WS+MTL+ALW+SGN

In [ ]:
exp_num = 3
epoch_n = 50

for i in range(exp_num):
    model = mymodel.AblationMark_Ws_Mtl_Alw_Sgns(in_seq_l, out_dim)
    score_train_vali, score_train_vali_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, epoch_n, trainset, valset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all, decay_params=[10,0.5])
    score_test, score_test_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, valset=testset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all)
    total_record = {'train_vali':score_train_vali, 'test':score_test,
                    'train_vali_clas':score_train_vali_clas, 'test_clas':score_test_clas}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset=dataset_name,model=type(model).__name__
                                                           ,appname='all',date=f'{file_code}-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)

In [ ]:
exp_num = 3
epoch_n = 50

for i in range(exp_num):
    model = mymodel.AblationMark_Ws_Mtl_Alw_Sgnh(in_seq_l, out_dim)
    score_train_vali, score_train_vali_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, epoch_n, trainset, valset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all, decay_params=[10,0.5])
    score_test, score_test_clas = util_md.train_val_Mark_DoubleTask_AutomaticWeightedLoss(device, model, valset=testset, loss_reg_factor=loss_factor,
                                         return_metric_reg_dict=metric_dict_one2all)
    total_record = {'train_vali':score_train_vali, 'test':score_test,
                    'train_vali_clas':score_train_vali_clas, 'test_clas':score_test_clas}
    filename = '{dataset}_{model}_{appname}_{date}'.format(dataset=dataset_name,model=type(model).__name__
                                                           ,appname='all',date=f'{file_code}-{i}')
    dm.save_dict(total_record, filename)
    mwm.save_model_weight(model, filename)